### Metadata source

In [3]:
import sys
import os
import pandas as pd
import json
from tqdm import tqdm
from dotenv import load_dotenv
from pydantic import BaseModel
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain_core.rate_limiters import InMemoryRateLimiter

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.articles import create_static_metadata
from src.llm import get_llm_client
from src.prompts import get_metadata_prompt

if not load_dotenv():
    raise Exception('Error loading .env file. Make sure to place a valid OPEN_AI_KEY in the .env file.')

In [5]:
rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.5,  # <-- Gemini Free Tier
    check_every_n_seconds=0.1,
)

llm_client = get_llm_client(
    # base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    max_tokens=1024,
    temperature=0.2,
    rate_limiter=rate_limiter,
)

ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'openai_api_key': None, ...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

Setup the paths to data sources

In [ ]:
ARTICLES_CLEAN_DIR = os.path.join("..", "data", "articles_clean")
METADATA_PATH = os.path.join("..", "data", "metadata.csv")

Extract metadata

In [ ]:
class ArticleTags(BaseModel):
    tags: list[str]

In [ ]:
def process_article(filename, with_tags=False):
    article_path = os.path.join(ARTICLES_CLEAN_DIR, filename)
    with open(article_path, "r", encoding="utf-8") as file:
        article = json.load(file)
    
    # Create static metadata for the article
    article_metadata = create_static_metadata(article, filename)
    
    if with_tags:
        # Prepare the prompt for the LLM using the article's text
        tags_prompt = get_metadata_prompt()
        query = tags_prompt.format(article_text=article["text"])
        
        # Invoke the LLM with structured output to extract tags
        llm = llm_client.with_structured_output(ArticleTags)
        response = llm.invoke([query])
        tags = response.tags
    else:
        tags = []
    
    article_metadata["tags"] = tags
    return article_metadata

In [ ]:
# List all cleaned article files
articles = os.listdir(ARTICLES_CLEAN_DIR)
metadata = []

# Adjust the max_workers based on your LLM rate limits and available resources
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(process_article, filename, with_tags=False): filename for filename in articles}
    for future in tqdm(as_completed(futures), total=len(futures)):
        try:
            result = future.result()
            metadata.append(result)
        except Exception as e:
            print(f"Error processing file {futures[future]}: {e}")


df_metadata = pd.DataFrame(metadata)
df_metadata.to_csv(METADATA_PATH, index=False)

In [ ]:
# Articles length statistics
df_metadata["words_count"].describe()

Categories by WZ

In [ ]:
df_metadata["category"].describe()

In [ ]:
df_metadata["category"].value_counts()

In [ ]:
# Tags statistics (only available if with_tags=True)
df_metadata.explode("tags")["tags"].value_counts()

In [ ]:
# Missing authors
df_metadata["author"].isnull().sum()

Metadata df

In [ ]:
df_metadata.head(5)